# TensorFlow 2 quickstart example

This short introduction uses [Keras](https://www.tensorflow.org/guide/keras/overview) to:

1. Build a neural network that classifies images.
2. Train this neural network.
3. And, finally, evaluate the accuracy of the model.

In [13]:
import tensorflow as tf

Load and prepare the [MNIST dataset](http://yann.lecun.com/exdb/mnist/). Convert the samples from integers to floating-point numbers:

In [14]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

Build the `tf.keras.Sequential` model by stacking layers. Choose an optimizer and loss function for training:

In [15]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10)
])

For each example the model returns a vector of "[logits](https://developers.google.com/machine-learning/glossary#logits)" or "[log-odds](https://developers.google.com/machine-learning/glossary#log-odds)" scores, one for each class.

In [16]:
predictions = model(x_train[:1]).numpy()
predictions


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



array([[ 0.10347158,  0.36183622, -0.31504995, -0.41068906,  0.33550173,
        -0.19566225,  0.6240908 , -0.1833655 , -0.14263046,  0.01133013]],
      dtype=float32)

The `tf.nn.softmax` function converts these logits to "probabilities" for each class: 

In [17]:
tf.nn.softmax(predictions).numpy()

array([[0.10329541, 0.13374802, 0.06797034, 0.0617709 , 0.1302718 ,
        0.07658943, 0.17385335, 0.07753704, 0.08076073, 0.09420295]],
      dtype=float32)

Note: It is possible to bake this `tf.nn.softmax` in as the activation function for the last layer of the network. While this can make the model output more directly interpretable, this approach is discouraged as it's impossible to
provide an exact and numerically stable loss calculation for all models when using a softmax output. 

The `losses.SparseCategoricalCrossentropy` loss takes a vector of logits and a `True` index and returns a scalar loss for each example.

In [18]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

This loss is equal to the negative log probability of the true class:
It is zero if the model is sure of the correct class.

This untrained model gives probabilities close to random (1/10 for each class), so the initial loss should be close to `-tf.log(1/10) ~= 2.3`.

In [19]:
loss_fn(y_train[:1], predictions).numpy()

2.5692961

In [20]:
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

The `Model.fit` method adjusts the model parameters to minimize the loss: 

In [21]:
model.fit(x_train, y_train, epochs=5)

Train on 60000 samples
Epoch 1/5
60000/60000 [==============================] - 3s 50us/sample - loss: 0.2992 - accuracy: 0.9134
Epoch 2/5
60000/60000 [==============================] - 3s 51us/sample - loss: 0.1463 - accuracy: 0.9564
Epoch 3/5
60000/60000 [==============================] - 3s 54us/sample - loss: 0.1092 - accuracy: 0.9671s - loss: 0.110
Epoch 4/5
60000/60000 [==============================] - 3s 54us/sample - loss: 0.0894 - accuracy: 0.9723
Epoch 5/5
60000/60000 [==============================] - 3s 56us/sample - loss: 0.0763 - accuracy: 0.9757


The `Model.evaluate` method checks the models performance, usually on a "[Validation-set](https://developers.google.com/machine-learning/glossary#validation-set)" or "[Test-set](https://developers.google.com/machine-learning/glossary#test-set)".

In [22]:
model.evaluate(x_test,  y_test, verbose=2)

10000/1 - 0s - loss: 0.0401 - accuracy: 0.9760


[0.07433162244707346, 0.976]

The image classifier is now trained to ~98% accuracy on this dataset. To learn more, read the [TensorFlow tutorials](https://www.tensorflow.org/tutorials/).

If you want your model to return a probability, you can wrap the trained model, and attach the softmax to it:

In [23]:
probability_model = tf.keras.Sequential([
  model,
  tf.keras.layers.Softmax()
])

In [24]:
probability_model(x_test[:5])

<tf.Tensor: id=60027, shape=(5, 10), dtype=float32, numpy=
array([[8.65259437e-07, 3.60588714e-08, 7.99128393e-05, 4.39942523e-04,
        2.48266274e-09, 1.09291761e-06, 1.27269514e-11, 9.98921156e-01,
        1.27754931e-04, 4.29317908e-04],
       [1.20998243e-06, 1.05037827e-04, 9.99864578e-01, 1.83671928e-05,
        1.47073100e-11, 1.85796239e-07, 7.91726507e-06, 9.76526429e-11,
        2.73457613e-06, 1.48666877e-11],
       [6.98746953e-06, 9.96970177e-01, 2.57668260e-04, 3.96416799e-05,
        1.43607511e-04, 3.73451476e-05, 3.26788548e-04, 8.95823003e-04,
        1.31397333e-03, 7.82218558e-06],
       [9.99821961e-01, 6.25724939e-09, 1.30656099e-05, 4.57728618e-07,
        2.85450324e-06, 3.32313857e-06, 9.30826136e-05, 9.47037097e-06,
        3.44671832e-07, 5.53688005e-05],
       [1.85280328e-06, 1.73743118e-08, 9.18435217e-06, 8.71062955e-09,
        9.95734513e-01, 3.19395258e-06, 1.66315624e-06, 6.72557362e-05,
        4.18651865e-07, 4.18184232e-03]], dtype=float32)>